In [1]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

In [2]:
#검색할 식당 데이터

df = pd.DataFrame(columns=['name', 'naverURL'])

items = ['만년곱창', '홍리마라탕', '히글리', '레모노이']

df['name'] = items
df

,name,naverURL
0,만년곱창,NaN
1,홍리마라탕,NaN
2,히글리,NaN
3,레모노이,NaN


In [15]:
# 식당 url 얻기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup

for i, keyword in enumerate(df['name'].tolist()):
    # 검색 url 만들기
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'  
    # 검색 url 접속 = 검색하기
    driver.get(naver_map_search_url)
    time.sleep(2) 
    # 검색 프레임 변경
    driver.switch_to.frame("searchIframe")
    time.sleep(1) 
     
    
    try:     
        #식당 정보가 있다면 첫번째 식당의 url을 가져오기
        
        if len(driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li')) != 0:   
            #식당 정보 클릭        
            driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.ouxiq > a:nth-child(1) > div").click()')
            time.sleep(2)
            
            # 검색한 플레이스의 개별 페이지 저장
            tmp = driver.current_url  
            res_code = re.findall(r"place/(\d+)", tmp)
            final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        
            print(final_url)
            df['naverURL'][i]=final_url 
        
    except: 
        df['naverURL'][i]= ''
        print('none') 
    
driver.close()
    #df.to_csv('url_completed.csv', encoding = 'utf-8-sig')

df

https://pcmap.place.naver.com/restaurant/1476611616/review/visitor#
none
https://pcmap.place.naver.com/restaurant/1887134163/review/visitor#
https://pcmap.place.naver.com/restaurant/1972327571/review/visitor#


,name,naverURL
0,만년곱창,https://pcmap.place.naver.com/restaurant/14766...
1,홍리마라탕,
2,히글리,https://pcmap.place.naver.com/restaurant/18871...
3,레모노이,https://pcmap.place.naver.com/restaurant/19723...


In [16]:
df['naverURL'][1]='https://pcmap.place.naver.com/restaurant/1478444688/review/visitor#'
df

,name,naverURL
0,만년곱창,https://pcmap.place.naver.com/restaurant/14766...
1,홍리마라탕,https://pcmap.place.naver.com/restaurant/14784...
2,히글리,https://pcmap.place.naver.com/restaurant/18871...
3,레모노이,https://pcmap.place.naver.com/restaurant/19723...


In [37]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()


count = 0 #
current = 0 #현재 진행 상황

goal = len(df['name']) #총 식당 수

#데이터 프레임으로 만들 빈 리스트 생성
rev_list=[]


for i in range(len(df)): 
    
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naverURL'][i]) 
    thisurl = df['naverURL'][i]
    time.sleep(2)
    
    #리뷰 더보기 버튼 다 누르기
    while True: 
        try: 
            time.sleep(1) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(3) 
            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').click() 
            
            time.sleep(2) 
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) 
            time.sleep(1) 
            
        except NoSuchElementException: 
            print('-모든 리뷰 더보기 완료-') 
            break 
    

     
        
    #리뷰 데이터 스크래핑을 위한 html 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        review_lists = soup.find_all('li', attrs = {'class':'YeINN'})
        print('총 리뷰 수 : ', len(review_lists))
        
        #리뷰 수가 0이 아닌 경우 리뷰 수집
        if len(review_lists) != 0 :
            
            for j, review in enumerate(review_lists):
                
                #내용 더보기가 있는 경우 내용 더보기를 눌러주기
                try:
                    if len(review.select(' div.ZZ4OK.IwhtZ > a > span.rvCSr')) != 0:
                        driver.execute_script('return document.querySelector("#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div > ul > li:nth-child(2) > div.ZZ4OK.IwhtZ > a > span.rvCSr").click()')
                        time.sleep(1)
                        
                        #리뷰 정보
                        user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                        print(user_review)
                            
                        #작성 날짜
                        date = review.select(' div.sb8UA > span:nth-child(1) > span:nth-child(3)')
                        print(date)
                            
                        #식당 평균 평점
                        #rating = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em')
                            
            
                        #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                        rev_list.append(
                        [
                            df['name'][i],
                            #rating.text,
                            user_review[0].text,
                            date[0].text
                        ]
                        )
                            
                        time.sleep(2)            
               
                
                
                except:
                    #리뷰 정보
                    user_review = review.select(' div.ZZ4OK.IwhtZ > a > span')
                    print(user_review)
                            
                    #작성 날짜
                    date = review.select(' div.sb8UA > span:nth-child(1) > span:nth-child(3)')
                    print(date)
                    
                    #식당 평균 평점
                    #rating = driver.find_element(by='xpath', value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em')
                    
    
                    #리뷰 정보가 있는 경우 식당 이름, 평점, 리뷰 텍스트, 작성 시간을 가져와서 데이터 프레임으로 만들기
                    rev_list.append(
                    [
                        df['name'][i],
                        #rating.text,
                        user_review[0].text,
                        date[0].text
                    ]
                    )            
        
                    time.sleep(2)                  
        
       
        


    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        #rate = driver.find_element(by='xpath',value = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[3]/span[1]/em').text       
        rev_list.append(
        [
            df['name'][i],
            #rate,
        ]
        ) 
        time.sleep(2)           
        print("리뷰가 없습니다")

    
            
    #검색한 창 닫고 검색 페이지로 돌아가기    
    # driver.close()
    # driver.switch_to.window(tabs[0])
    print("기본 페이지로 돌아가기")

        
driver.close()

#스크래핑한 데이터를 데이터 프레임으로 만들기
#평점이 없는 경우를 위해 평점 정보 삭제    
column = ["name", "review", "timestamp"]
df2 = pd.DataFrame(rev_list, columns=column)
df2   

진행상황 : 1/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  3
[<span class="zPfVt">막창양념구이 먹었어요! 처음 생긴곳이라 리뷰가없어서 걱정하면서 먹었는데, 너무 맛있어서 좋았습니다 ㅎㅎ 사장님도 친절하셔서 기분좋았어요! 배민으로 2만원이상 포장하면 3천원할인된다고도 알려주셨어요. 서비스도 푸
짐해서 또한번놀랐어요.  넘감사해서 리뷰적을곳을 찾아서 적습니다. 잘먹었습니다. 번창하세요!! :)</span>, <span class="rvCSr"><svg aria-hidden="true" class="Ky28p" viewbox="0 0 12 7" xmlns="http://www.w3.org/2000/svg"><path d="M11.47.52a.74.74 0 00-1.04 0l-4.4 4.45v.01L1.57.52A.74.74 0 10.53 1.57l5.12 5.08a.5.5 0 00.7 0l5.12-5.08a.74.74 0 000-1.05z"></path></svg><span class="place_blind">내용 더보기</span></span>]
[<span class="place_blind">2022년 5월 12일 목요일</span>]
기본 페이지로 돌아가기
진행상황 : 2/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  10
기본 페이지로 돌아가기
진행상황 : 3/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  22
기본 페이지로 돌아가기
진행상황 : 4/4
-모든 리뷰 더보기 완료-
총 리뷰 수 :  4
[<span class="zPfVt">맛있다고 소문듣고 퇴근후 찾아온 디저트카페!! 들어오자마자 낯선 이름의 휘낭시에 '치즈올리브휘낭시에' 모험심으로 도전ㅋㅋㅋ 낯선 첫인상이지만 진짜 맛있었음! 단짠단짠 👍👍 다른 디저트는 다 포장했어용 !! 내일 하나하
나 다먹어야겠어용 😃 바나나카라멜도 인스타에서 너무 궁금했는데 잘먹겠습니다 ❣️</span>, <span class="rvCSr"><svg aria-hidden="true" class

,name,review,timestamp
0,만년곱창,"막창양념구이 먹었어요! 처음 생긴곳이라 리뷰가없어서 걱정하면서 먹었는데, 너무 맛있...",2022년 5월 12일 목요일
1,레모노이,맛있다고 소문듣고 퇴근후 찾아온 디저트카페!! 들어오자마자 낯선 이름의 휘낭시에 '...,2022년 10월 2일 일요일
